#### Python's requests library-> get() function will get web page. The response r contains many things but uisng  r.content we can get HTML .

In [27]:
import requests

url = "https://www.allsides.com/media-bias/media-bias-ratings"

r = requests.get(url)

print(r.content[:100])

b'<!DOCTYPE html>\n<html  lang="en" dir="ltr" prefix="content: http://purl.org/rss/1.0/modules/content/'


#### Parsing HTML with BeautifulSoup

Requests used for get the web page.
BeautifulSoup use for parse HTML and XML .
When we pass our HTML to the BeautifulSoup constructor we get an object in return that we can then navigate like the original tree structure of the DOM.

In [28]:
#creating a new soup object
from bs4 import BeautifulSoup

soup = BeautifulSoup(r.content, 'html.parser')

#### Finding elements and data

select_one -> returns single element
select -> returns list of elements
Ex: 
1. select_one("a") -> gets an anchor/link element
2. select_one('body') -> gets the body element
3. select_one('.temp') -> to get <a class="temp"></a>
4. select_one('#temp') -> to get <a id="temp"></a> 
5. select_one('.temp.example') -> to get <div class="temp"><a></a></div>
6. select_one('.temp .example') -> o get <div class="temp"><a class="example"></a></div> (gain, note the space between .temp and .example. The space tells the selector that the class after the space is a child of the class before the space.)


select all the rows from the table and then parse out the information from each row.

In [29]:
#get each row select all tr inside tbody
rows = soup.select('tbody tr')# extract all <tr> tags that are childern of parent<tbody> tag

In [30]:
#extract news source name
row = rows[0]

name = row.select_one('.views-field-title').text.strip()

print(name)

Associated Press


In [31]:
#extrcat source page link
link_als = row.select_one('.views-field-title a')['href']
link_als = "https://www.allsides.com" + link_als

print(link_als)

https://www.allsides.com/news-source/associated-press-media-bias


In [32]:
#extract bias rating
bias = row.select_one('.views-field-field-bias-image a')['href']
bias = bias.split('/')[-1]

print(bias)

left-center


In [33]:
#extract community feedback data
agree = row.select_one('.agree').text
agree = int(agree)

disagree = row.select_one('.disagree').text
disagree = int(disagree)

agree_ratio = agree / disagree

print(f"Agree: {agree}, Disagree: {disagree}, Ratio {agree_ratio:.2f}")

Agree: 31601, Disagree: 24694, Ratio 1.28


In [34]:
print(row.select_one('.community-feedback-rating-page'))

None


In [35]:
def get_agreeance_text(ratio):
    if ratio > 3: return "absolutely agrees"
    elif 2 < ratio <= 3: return "strongly agrees"
    elif 1.5 < ratio <= 2: return "agrees"
    elif 1 < ratio <= 1.5: return "somewhat agrees"
    elif ratio == 1: return "neutral"
    elif 0.67 < ratio < 1: return "somewhat disagrees"
    elif 0.5 < ratio <= 0.67: return "disagrees"
    elif 0.33 < ratio <= 0.5: return "strongly disagrees"
    elif ratio <= 0.33: return "absolutely disagrees"
    else: return None
    
print(get_agreeance_text(2.5))

strongly agrees


#### gets data from every row on the first page:

In [36]:

data= []

for row in rows:
    d = dict()
    
    d['name'] = row.select_one('.views-field-title').text.strip()
    d['allsides_page'] = 'https://www.allsides.com' + row.select_one('.views-field-title a')['href']
    d['bias'] = row.select_one('.views-field-field-bias-image a')['href'].split('/')[-1]
    d['agree'] = int(row.select_one('.agree').text)
    d['disagree'] = int(row.select_one('.disagree').text)
    d['agree_ratio'] = d['agree'] / d['disagree']
    d['agreeance_text'] = get_agreeance_text(d['agree_ratio'])
    
    data.append(d)
    
    

In [37]:
print(data[0])

{'name': 'Associated Press', 'allsides_page': 'https://www.allsides.com/news-source/associated-press-media-bias', 'bias': 'left-center', 'agree': 31601, 'disagree': 24694, 'agree_ratio': 1.2797035717178262, 'agreeance_text': 'somewhat agrees'}


In [38]:
for i in range(len(data)):
    print(data[i])

{'name': 'Associated Press', 'allsides_page': 'https://www.allsides.com/news-source/associated-press-media-bias', 'bias': 'left-center', 'agree': 31601, 'disagree': 24694, 'agree_ratio': 1.2797035717178262, 'agreeance_text': 'somewhat agrees'}
{'name': 'Associated Press Fact Check', 'allsides_page': 'https://www.allsides.com/news-source/ap-fact-check-media-bias', 'bias': 'left-center', 'agree': 5960, 'disagree': 6811, 'agree_ratio': 0.8750550579944207, 'agreeance_text': 'somewhat disagrees'}
{'name': 'CNN (Online News)', 'allsides_page': 'https://www.allsides.com/news-source/cnn-media-bias', 'bias': 'left', 'agree': 58489, 'disagree': 50574, 'agree_ratio': 1.1565033416379957, 'agreeance_text': 'somewhat agrees'}
{'name': 'Fox News (Online News)', 'allsides_page': 'https://www.allsides.com/news-source/fox-news-media-bias', 'bias': 'right', 'agree': 49093, 'disagree': 51436, 'agree_ratio': 0.954448246364414, 'agreeance_text': 'somewhat disagrees'}
{'name': 'Insider', 'allsides_page': 'ht